# Tables and numbers for Keizer et al. (2023): "The acceleration of sea-level rise along the coast of the Netherlands started in the 1960s"
Read output data from gam_model.ipynb

__Note on the environment required to run the notebook:__

An installation of Python 3.9 with recent versions of the libraries (numpy, scipy, matplotlib, pandas, xarray, statsmodels) should run this Notebook wihout modification.


### Overview of this notebook:
1. Model Evaluation (incl. Table 1)
2. Wind influence on sea level
3. Sea Level Rates (incl. Table 2)
4. Hypothesis testing (incl. Table 3)

In [75]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import norm, linregress

import local_functions as loc

# For higher resolution images on retina display screens
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')

In [76]:
region = 'Netherlands' # Choose from 'Netherlands', 'Belgium', 'Vlissingen', 'Hoek van Holland', 'Den Helder', 'Delfzijl', 'Harlingen', 'IJmuiden'
bootsize = 10000 # Choose from 100, 1000, 10000
output_dir = f'./outputs/{region}/'

if os.path.exists(output_dir)==False:
    os.makedirs(output_dir)

if os.path.exists(output_dir+str(bootsize))==False:
    os.makedirs(output_dir+str(bootsize))

## 1. Model Evaluation

### 1.1 Table 1: Summary table of model performance

In [77]:
resm_df = pd.read_csv(f'{output_dir}{bootsize}/table_model_performance.csv')

In [78]:
resm_df = pd.read_csv(f'{output_dir}{bootsize}/table_model_performance.csv')
resm_df = resm_df.iloc[[0,1,3,4]]
resm_df.set_index('Statistical model', inplace=True)
resm_df['Model equation'] = ['$\eta_i=  \sum_j \alpha_{j} \phi_{ji}$', 
                             '$\eta_i = \sum_j \alpha_{j} \phi_{ji} + \beta_{1} \cos(2\pi i /18.613) + \beta_{2} \sin(2\pi i /18.613)$', 
                             '$\eta_i = \sum_j \alpha_{j} \phi_{ji} + \beta_{1} \cos(2\pi i /18.613) + \beta_{2} \sin(2\pi i /18.613)  +  \beta_{3} |u_i|u_i$', 
                             '$\eta_i = \sum_j \alpha_{j} \phi_{ji} + \beta_{1} \cos(2\pi i /18.613) + \beta_{2} \sin(2\pi i /18.613)  +  \beta_{4} \Delta p_i$']
resm_df = resm_df[['Model equation', 'Degrees of freedom', 'Deviance']]

In [79]:
resm_df

,Model equation,Degrees of freedom,Deviance
Statistical model,,,
Tr : Trend only,$\eta_i= \sum_j lpha_{j} \phi_{ji}$,4.7,1167.0
TrNc : Trend and nodal tide,$\eta_i = \sum_j lpha_{j} \phi_{ji} + eta_{1...,6.6,1033.0
"TrNcW : Trend, nodal tide and wind (zonal&meridional velocity)",$\eta_i = \sum_j lpha_{j} \phi_{ji} + eta_{1...,8.6,423.0
"TrNcPd : Trend, nodal tide and wind (pressure difference)",$\eta_i = \sum_j lpha_{j} \phi_{ji} + eta_{1...,7.6,652.0


Export to latex table format to copy paste directly to the paper

In [80]:
print(resm_df.to_latex())

\begin{tabular}{llrr}
\toprule
{} &                                     Model equation &  Degrees of freedom &  Deviance \\
Statistical model                                  &                                                    &                     &           \\
\midrule
Tr : Trend only                                    &              \$\textbackslash eta\_i=  \textbackslash sum\_j lpha\_\{j\} \textbackslash phi\_\{ji\}\$ &                 4.7 &    1167.0 \\
TrNc : Trend and nodal tide                        &  \$\textbackslash eta\_i = \textbackslash sum\_j lpha\_\{j\} \textbackslash phi\_\{ji\} +eta\_\{1... &                 6.6 &    1033.0 \\
TrNcW : Trend, nodal tide and wind (zonal\&merid... &  \$\textbackslash eta\_i = \textbackslash sum\_j lpha\_\{j\} \textbackslash phi\_\{ji\} +eta\_\{1... &                 8.6 &     423.0 \\
TrNcPd : Trend, nodal tide and wind (pressure d... &  \$\textbackslash eta\_i = \textbackslash sum\_j lpha\_\{j\} \textbackslash phi\_\{ji\} +eta\_

/var/folders/sg/bjgd139d48v526p07tp867mh0000gn/T/ipykernel_37633/3890745105.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(resm_df.to_latex())


### 1.2 Also compute some percentages for the paper

Deviance

In [81]:
print(round(100-100*resm_df.Deviance[1]/resm_df.Deviance[0],0))
print(round(100-100*resm_df.Deviance[2]/resm_df.Deviance[1],0))
print(round(100-100*resm_df.Deviance[3]/resm_df.Deviance[1],0))

11.0
59.0
37.0


## 2. Wind contribution to sea level

In [82]:
wind_rec = pd.read_csv(f'{output_dir}/WindInfluence.csv')
wind_rec.set_index('time', inplace=True)

In [83]:
# Estimate the linear trend for different periods

mid_point = int(len(wind_rec.index)/2)

lin_trend_na1 = round(linregress(wind_rec.index[0:mid_point], 
                                 wind_rec['NearestPointAverage2_trend'].iloc[0:mid_point]).slope*10, 2)
lin_trend_na2 = round(linregress(wind_rec.index[mid_point:-1], 
                                 wind_rec['NearestPointAverage2_trend'].iloc[mid_point:-1]).slope*10, 2)

lin_trend_pd1 = round(linregress(wind_rec.index[0:mid_point], 
                                 wind_rec['PressureDifference_trend'].iloc[0:mid_point]).slope*10, 2)
lin_trend_pd2 = round(linregress(wind_rec.index[mid_point:-1], 
                                 wind_rec['PressureDifference_trend'].iloc[mid_point:-1]).slope*10, 2)


In [84]:
lin_trend_na1, lin_trend_na2, lin_trend_pd1, lin_trend_pd2

(0.17, 0.13, 0.42, 0.14)

In [85]:
# Standard error in the estimation of the linear trend

lin_trend_se_na1 = round(linregress(wind_rec.index[0:mid_point], 
                                 wind_rec['NearestPointAverage2_trend'].iloc[0:mid_point]).stderr*10, 3)
lin_trend_se_na2 = round(linregress(wind_rec.index[mid_point:-1], 
                                 wind_rec['NearestPointAverage2_trend'].iloc[mid_point:-1]).stderr*10, 3)

lin_trend_se_pd1 = round(linregress(wind_rec.index[0:mid_point], 
                                 wind_rec['PressureDifference_trend'].iloc[0:mid_point]).stderr*10, 3)
lin_trend_se_pd2 = round(linregress(wind_rec.index[mid_point:-1], 
                                 wind_rec['PressureDifference_trend'].iloc[mid_point:-1]).stderr*10, 3)

In [86]:
lin_trend_se_na1, lin_trend_se_na2, lin_trend_se_pd1, lin_trend_se_pd2

(0.001, 0.0, 0.01, 0.003)

These standard errors are very small. The uncertainty in estimating the trend in wind influence on sea level does not come from fitting a linear trend to a wind time series (which these standard errors measure), it comes from the wind data and the method itself. 

Therefore the uncertainty is better estimated by looking at the difference between two methods which is what we do in the paper.

## 3. Rates of sea level rise

### 3.1 Read rates data

In [87]:
# tg_df = pd.read_csv(f'{output_dir}data_tide_gauges.csv')

# res_nr = pd.read_csv(f'{output_dir}results_NoRegressor.csv')
# res_nt = pd.read_csv(f'{output_dir}results_NodalTide.csv')
# res_na = pd.read_csv(f'{output_dir}results_NearestPointAverage.csv')
# res_pd = pd.read_csv(f'{output_dir}results_PressureDifference.csv')

# for df in [tg_df, res_nr, res_nt, res_na, res_pd]:
#     df.set_index('time', inplace=True)

In [88]:
rates_nr = pd.read_csv(f'{output_dir}{bootsize}/rates_NoRegressor.csv')
rates_nt = pd.read_csv(f'{output_dir}{bootsize}/rates_NodalTide.csv')
rates_na = pd.read_csv(f'{output_dir}{bootsize}/rates_NearestPointAverage2.csv')
rates_pd = pd.read_csv(f'{output_dir}{bootsize}/rates_PressureDifference.csv')

rates_q_nr = pd.read_csv(f'{output_dir}{bootsize}/rates_quantiles_NoRegressor.csv')
rates_q_nt = pd.read_csv(f'{output_dir}{bootsize}/rates_quantiles_NodalTide.csv')
rates_q_na = pd.read_csv(f'{output_dir}{bootsize}/rates_quantiles_NearestPointAverage2.csv')
rates_q_pd = pd.read_csv(f'{output_dir}{bootsize}/rates_quantiles_PressureDifference.csv')

for df in [rates_nr, rates_nt, rates_na, rates_pd,
           rates_q_nr, rates_q_nt, rates_q_na, rates_q_pd,]:
    df.set_index('time', inplace=True)

In [89]:
# Compute the standard errors in rates

ser_nr = rates_nr.std(axis=1)
ser_nt = rates_nt.std(axis=1)
ser_na = rates_na.std(axis=1)
ser_pd = rates_pd.std(axis=1)

In [90]:
long_names = ['Trend only', 
              'Trend and nodal tide', 
              'Trend, nodal tide and wind (velocity)', 
              'Trend, nodal tide and wind (pressure)']

short_names = ['Tr', 'TrNc', 'TrNcW', 'TrNcPd']

comb_names = [f'{short_names[i]} : {long_names[i]}' for i in range(4)]

In the paper 0.00 is replaced by <0.01

### 3.2 Print the rate numbers and uncertainty range

In [91]:
a = [1,2]
print(a[1:2])

[2]


In [92]:
print('Format: Median [0.05, 0.95]')

list_st = [1900, 1920, 1940, 1960, 1980, 2000, 1890, 1960, 1890]
list_pr = [19, 19, 19, 19, 19, 19, 69, 61, 131]

for idx, res in enumerate([rates_q_nr, rates_q_nt, rates_q_na, rates_q_pd]):
    print('')
    print(f'Model: {short_names[idx]}')
    
    trend_lst = []
    if idx == 0:
        period_lst = []
          
    for idx2, st in enumerate(list_st):
        avg_rate = round(res.loc[st:st+list_pr[idx2],'0.5'].mean(axis=0),1)
        low_bound = round(res.loc[st:st+list_pr[idx2],'0.05'].mean(axis=0),1)
        high_bound = round(res.loc[st:st+list_pr[idx2],'0.95'].mean(axis=0),1)
        
        print(f'Period: {st}-{st+list_pr[idx2]}')
        print(f'{avg_rate} [{low_bound},{high_bound}]')
        
        if idx == 0:
            period_lst.append(f'{st}-{st+list_pr[idx2]}')
        
        trend_lst.append(f'{avg_rate} [{low_bound},{high_bound}]')
        
    if idx == 0:
        df = pd.DataFrame({'Period': period_lst}).set_index('Period')
        
    df[short_names[idx]] = trend_lst
        
table_rates = df.T

Format: Median [0.05, 0.95]

Model: Tr
Period: 1900-1919
2.2 [1.4,2.9]
Period: 1920-1939
1.7 [1.1,2.3]
Period: 1940-1959
1.6 [1.0,2.3]
Period: 1960-1979
1.8 [1.2,2.5]
Period: 1980-1999
2.3 [1.6,2.9]
Period: 2000-2019
2.7 [1.8,3.7]
Period: 1890-1959
1.9 [1.1,2.6]
Period: 1960-2021
2.3 [1.5,3.0]
Period: 1890-2021
2.1 [1.3,2.8]

Model: TrNc
Period: 1900-1919
2.3 [1.8,2.8]
Period: 1920-1939
1.7 [1.3,2.1]
Period: 1940-1959
1.6 [1.2,2.0]
Period: 1960-1979
1.8 [1.4,2.2]
Period: 1980-1999
2.3 [1.9,2.7]
Period: 2000-2019
2.7 [2.1,3.4]
Period: 1890-1959
1.9 [1.5,2.4]
Period: 1960-2021
2.3 [1.8,2.8]
Period: 1890-2021
2.1 [1.6,2.6]

Model: TrNcW
Period: 1900-1919
1.7 [1.3,2.2]
Period: 1920-1939
1.7 [1.4,2.1]
Period: 1940-1959
1.5 [1.2,1.9]
Period: 1960-1979
1.4 [1.1,1.7]
Period: 1980-1999
2.2 [1.9,2.6]
Period: 2000-2019
2.9 [2.4,3.5]
Period: 1890-1959
1.7 [1.2,2.1]
Period: 1960-2021
2.2 [1.8,2.6]
Period: 1890-2021
1.9 [1.5,2.3]

Model: TrNcPd
Period: 1900-1919
1.9 [1.3,2.4]
Period: 1920-1939
1.8 [

In [93]:
# changed format

print('Format: unc{Median}{0.05}{0.95}')

for idx, res in enumerate([rates_q_nr, rates_q_nt, rates_q_na, rates_q_pd]):
    print('')
    print(f'Model: {short_names[idx]}')
    
    trend_lst = []
    if idx == 0:
        period_lst = []
          
    for idx2, st in enumerate(list_st):
        avg_rate = round(res.loc[st:st+list_pr[idx2],'0.5'].mean(axis=0),1)
        low_bound = round(res.loc[st:st+list_pr[idx2],'0.05'].mean(axis=0),1)
        high_bound = round(res.loc[st:st+list_pr[idx2],'0.95'].mean(axis=0),1)
        
        # print(f'Period: {st}-{st+list_pr[idx2]}')
        # print(f'{avg_rate} [{low_bound},{high_bound}]')
        
        if idx == 0:
            y1 = f'{st}'
            y2 = f'{st+list_pr[idx2]}'
            if y1[:2]==y2[:2]:
                y2 = y2[2:]
            
            period_lst.append(f'{y1}--{y2}')
        
        trend_lst.append(f'\\unc{{{avg_rate}}}{{{low_bound}}}{{{high_bound}}}')
        
    if idx == 0:
        df_ = pd.DataFrame({'Period': period_lst}).set_index('Period')
        
    df_[short_names[idx]] = trend_lst
        
table_rates_ = df_.T

Format: unc{Median}{0.05}{0.95}

Model: Tr

Model: TrNc

Model: TrNcW

Model: TrNcPd


### 3.3 Table 2: Trend values for different time periods
Make table containing all different trends mentioned in the paper

For the 4 GAMs: Tr, TrNc, TrNcZw, TrNcPd

For the sea-level trend: 1900-1919, 1920-1939, 1940-1959, 1960-1979, 1980-1999, 2000-2019, 1890-2019, 1890-1960s 1960s-2019

For the wind-driven sea-level: whole period, two half periods. Where to add this?

In [94]:
table_rates

Period,1900-1919,1920-1939,1940-1959,1960-1979,1980-1999,2000-2019,1890-1959,1960-2021,1890-2021
Tr,"2.2 [1.4,2.9]","1.7 [1.1,2.3]","1.6 [1.0,2.3]","1.8 [1.2,2.5]","2.3 [1.6,2.9]","2.7 [1.8,3.7]","1.9 [1.1,2.6]","2.3 [1.5,3.0]","2.1 [1.3,2.8]"
TrNc,"2.3 [1.8,2.8]","1.7 [1.3,2.1]","1.6 [1.2,2.0]","1.8 [1.4,2.2]","2.3 [1.9,2.7]","2.7 [2.1,3.4]","1.9 [1.5,2.4]","2.3 [1.8,2.8]","2.1 [1.6,2.6]"
TrNcW,"1.7 [1.3,2.2]","1.7 [1.4,2.1]","1.5 [1.2,1.9]","1.4 [1.1,1.7]","2.2 [1.9,2.6]","2.9 [2.4,3.5]","1.7 [1.2,2.1]","2.2 [1.8,2.6]","1.9 [1.5,2.3]"
TrNcPd,"1.9 [1.3,2.4]","1.8 [1.3,2.2]","1.5 [1.1,2.0]","1.5 [1.0,1.9]","2.1 [1.6,2.6]","2.7 [2.0,3.4]","1.8 [1.2,2.3]","2.1 [1.6,2.7]","1.9 [1.4,2.5]"


In [95]:
print(table_rates.to_latex())

\begin{tabular}{llllllllll}
\toprule
Period &      1900-1919 &      1920-1939 &      1940-1959 &      1960-1979 &      1980-1999 &      2000-2019 &      1890-1959 &      1960-2021 &      1890-2021 \\
\midrule
Tr     &  2.2 [1.4,2.9] &  1.7 [1.1,2.3] &  1.6 [1.0,2.3] &  1.8 [1.2,2.5] &  2.3 [1.6,2.9] &  2.7 [1.8,3.7] &  1.9 [1.1,2.6] &  2.3 [1.5,3.0] &  2.1 [1.3,2.8] \\
TrNc   &  2.3 [1.8,2.8] &  1.7 [1.3,2.1] &  1.6 [1.2,2.0] &  1.8 [1.4,2.2] &  2.3 [1.9,2.7] &  2.7 [2.1,3.4] &  1.9 [1.5,2.4] &  2.3 [1.8,2.8] &  2.1 [1.6,2.6] \\
TrNcW  &  1.7 [1.3,2.2] &  1.7 [1.4,2.1] &  1.5 [1.2,1.9] &  1.4 [1.1,1.7] &  2.2 [1.9,2.6] &  2.9 [2.4,3.5] &  1.7 [1.2,2.1] &  2.2 [1.8,2.6] &  1.9 [1.5,2.3] \\
TrNcPd &  1.9 [1.3,2.4] &  1.8 [1.3,2.2] &  1.5 [1.1,2.0] &  1.5 [1.0,1.9] &  2.1 [1.6,2.6] &  2.7 [2.0,3.4] &  1.8 [1.2,2.3] &  2.1 [1.6,2.7] &  1.9 [1.4,2.5] \\
\bottomrule
\end{tabular}



/var/folders/sg/bjgd139d48v526p07tp867mh0000gn/T/ipykernel_37633/1721568710.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table_rates.to_latex())


In [96]:
table_rates_

Period,1900--19,1920--39,1940--59,1960--79,1980--99,2000--19,1890--1959,1960--2021,1890--2021
Tr,\unc{2.2}{1.4}{2.9},\unc{1.7}{1.1}{2.3},\unc{1.6}{1.0}{2.3},\unc{1.8}{1.2}{2.5},\unc{2.3}{1.6}{2.9},\unc{2.7}{1.8}{3.7},\unc{1.9}{1.1}{2.6},\unc{2.3}{1.5}{3.0},\unc{2.1}{1.3}{2.8}
TrNc,\unc{2.3}{1.8}{2.8},\unc{1.7}{1.3}{2.1},\unc{1.6}{1.2}{2.0},\unc{1.8}{1.4}{2.2},\unc{2.3}{1.9}{2.7},\unc{2.7}{2.1}{3.4},\unc{1.9}{1.5}{2.4},\unc{2.3}{1.8}{2.8},\unc{2.1}{1.6}{2.6}
TrNcW,\unc{1.7}{1.3}{2.2},\unc{1.7}{1.4}{2.1},\unc{1.5}{1.2}{1.9},\unc{1.4}{1.1}{1.7},\unc{2.2}{1.9}{2.6},\unc{2.9}{2.4}{3.5},\unc{1.7}{1.2}{2.1},\unc{2.2}{1.8}{2.6},\unc{1.9}{1.5}{2.3}
TrNcPd,\unc{1.9}{1.3}{2.4},\unc{1.8}{1.3}{2.2},\unc{1.5}{1.1}{2.0},\unc{1.5}{1.0}{1.9},\unc{2.1}{1.6}{2.6},\unc{2.7}{2.0}{3.4},\unc{1.8}{1.2}{2.3},\unc{2.1}{1.6}{2.7},\unc{1.9}{1.4}{2.5}


In [97]:
print(table_rates_.to_latex())

\begin{tabular}{llllllllll}
\toprule
Period &             1900--19 &             1920--39 &             1940--59 &             1960--79 &             1980--99 &             2000--19 &           1890--1959 &           1960--2021 &           1890--2021 \\
\midrule
Tr     &  \textbackslash unc\{2.2\}\{1.4\}\{2.9\} &  \textbackslash unc\{1.7\}\{1.1\}\{2.3\} &  \textbackslash unc\{1.6\}\{1.0\}\{2.3\} &  \textbackslash unc\{1.8\}\{1.2\}\{2.5\} &  \textbackslash unc\{2.3\}\{1.6\}\{2.9\} &  \textbackslash unc\{2.7\}\{1.8\}\{3.7\} &  \textbackslash unc\{1.9\}\{1.1\}\{2.6\} &  \textbackslash unc\{2.3\}\{1.5\}\{3.0\} &  \textbackslash unc\{2.1\}\{1.3\}\{2.8\} \\
TrNc   &  \textbackslash unc\{2.3\}\{1.8\}\{2.8\} &  \textbackslash unc\{1.7\}\{1.3\}\{2.1\} &  \textbackslash unc\{1.6\}\{1.2\}\{2.0\} &  \textbackslash unc\{1.8\}\{1.4\}\{2.2\} &  \textbackslash unc\{2.3\}\{1.9\}\{2.7\} &  \textbackslash unc\{2.7\}\{2.1\}\{3.4\} &  \textbackslash unc\{1.9\}\{1.5\}\{2.4\} &  \textbackslash unc\{2.3\}\{1.

/var/folders/sg/bjgd139d48v526p07tp867mh0000gn/T/ipykernel_37633/2649762718.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(table_rates_.to_latex())


#### post-processing steps of string
1. in the above output `\textbackslash unc` needs to be replaced by `\unc`, and both curly bracket need to be un-escaped, i.e. `\{` is replaced by `{` and the same for `}`
2. the short model name should be italizised: e.g., `Tr` to `\textit{Tr}`
3. the columns look better when centered: so `\begin{tabular}{llllllllll}` becomes `\begin{tabular}{lccccccccc}`

## 4. Hypothesis testing: compute some p-values

### Hypothesis that the rates are the same at different years

In [98]:
t1 = 2019
all_t0 = [1900, 1960]

list_rates = [rates_nr, rates_nt, rates_na, rates_pd]
list_rates_q = [rates_q_nr, rates_q_nt, rates_q_na, rates_q_pd]

col_names = [f'p-values [r({t1}) , r({y})]' for y in all_t0]

res_ar = np.zeros([4, 2])

for idt, t0 in enumerate(all_t0):
    for idd, (rates, rates_q) in enumerate(zip(list_rates, list_rates_q)):

        sigma = np.std(rates.loc[t1] - rates.loc[t0])

        # observed difference in rate
        drate = rates_q['0.5'].loc[t1] - rates_q['0.5'].loc[t0]

        # p-value is the probability that the estimated rate difference would exceed  
        # the value drate if the true rates were equal
        # (this looks like your formula!)
        res_ar[idd, idt] = round(1-norm.cdf(drate/sigma), 2)
    
res_df = pd.DataFrame(res_ar, 
                      index=short_names, 
                      columns=col_names)

res_df.index.name = 'Statistical model'

In [99]:
res_df

,"p-values [r(2019) , r(1900)]","p-values [r(2019) , r(1960)]"
Statistical model,,
Tr,0.25,0.08
TrNc,0.29,0.02
TrNcW,0.00,0.00
TrNcPd,0.12,0.02


### Hypothesis that the rates are the same at different periods

In [100]:
t1 = 2000
all_t0 = [1900, 1920, 1940]

list_rates = [rates_nr, rates_nt, rates_na, rates_pd]
list_rates_q = [rates_q_nr, rates_q_nt, rates_q_na, rates_q_pd]

col_names = [f'$r_{{{t1}-{t1+19}}}$ vs $r_{{{y}-{y+19}}}$' for y in all_t0]

res_ar = np.zeros([4, 3])

for idt, t0 in enumerate(all_t0):
    for idd, (rates, rates_q) in enumerate(zip(list_rates, list_rates_q)):
        sigma = np.std(rates.loc[t1: t1+19].mean(axis=0) - 
                       rates.loc[t0: t0+19].mean(axis=0))

        # observed difference in rate
        drate = (rates_q['0.5'].loc[t1: t1+19].mean(axis=0) - 
                 rates_q['0.5'].loc[t0: t0+19].mean(axis=0))

        # p-value is the probability that the estimated rate difference would exceed  
        # the value drate if the true rates were equal
        res_ar[idd, idt] = round(1-norm.cdf(drate/sigma), 2)
    
res_df = pd.DataFrame(res_ar, 
                      index=short_names, 
                      columns=col_names)

res_df.index.name = 'Statistical model'

In [101]:
res_df

,$r_{2000-2019}$ vs $r_{1900-1919}$,$r_{2000-2019}$ vs $r_{1920-1939}$,$r_{2000-2019}$ vs $r_{1940-1959}$
Statistical model,,,
Tr,0.23,0.05,0.06
TrNc,0.15,0.01,0.01
TrNcW,0.00,0.00,0.00
TrNcPd,0.05,0.02,0.01


In [102]:
res_df_100 = res_df*100  # Show results in percentages

In [103]:
res_df_100

,$r_{2000-2019}$ vs $r_{1900-1919}$,$r_{2000-2019}$ vs $r_{1920-1939}$,$r_{2000-2019}$ vs $r_{1940-1959}$
Statistical model,,,
Tr,23.0,5.0,6.0
TrNc,15.0,1.0,1.0
TrNcW,0.0,0.0,0.0
TrNcPd,5.0,2.0,1.0


In [104]:
print(res_df_100.to_latex)

<bound method NDFrame.to_latex of                    $r_{2000-2019}$ vs $r_{1900-1919}$  \
Statistical model                                       
Tr                                               23.0   
TrNc                                             15.0   
TrNcW                                             0.0   
TrNcPd                                            5.0   

                   $r_{2000-2019}$ vs $r_{1920-1939}$  \
Statistical model                                       
Tr                                                5.0   
TrNc                                              1.0   
TrNcW                                             0.0   
TrNcPd                                            2.0   

                   $r_{2000-2019}$ vs $r_{1940-1959}$  
Statistical model                                      
Tr                                                6.0  
TrNc                                              1.0  
TrNcW                                             0.0  
